In [101]:
### STEP 2 ###
from glob import glob
import os
import librosa
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

loc = './pr_lab1_2020-21_data/digits'

'''
parse_free_digits(loc)'''

def data_parser(directory):
    files = glob(os.path.join(directory, "*.wav"))
    fnames = [f.split('/')[3].split('.')[0] for f in files]
    ## extract ids from filename
    ids = [int(f[-2]+f[-1]) if f[-2].isdigit() else int(f[-1]) for f in fnames]
    ## extract float arrays of signals
    waves = [librosa.load(fpath,sr=None)[0] for fpath in files]
    ## extract digits from filenames
    digits = []
    for i in range(len(fnames)):
        if str(ids[i]) in fnames[i]:
            fnames[i] = fnames[i].replace(str(ids[i]),'')
    return waves, ids, fnames


In [164]:
### STEP 3 ###

waves,_,_ = data_parser(loc)

def extract_features(waves,num_mfcc=13,window=25,step=10):
    '''
    returns list of np.arrays of shape (frames of signal x num_mfcc)
    '''
    w = window*16000//1000 ## length of window in Hz
    s = step*16000//1000  ## length of step in Hz
    mfccs = [
        librosa.feature.mfcc(y=wav,n_mfcc=num_mfcc,n_fft=w,hop_length=w-s).T ## by default returns (MFCC x Waves) shape
        for wav in tqdm(waves, desc='Extracting MFCC features...')
    ]
    deltas = [librosa.feature.delta(mfcc) for mfcc in mfccs]
    ddeltas = [librosa.feature.delta(d) for d in mfccs]
    
    #print(mfccs[0].shape,deltas[0].shape,ddeltas[0].shape)
    
    print(f'Feature extraction completed: {num_mfcc} MFCC per frame.')
    return mfccs, deltas, ddeltas

#extract_features(waves)


In [152]:
### STEP 4 ### (pain in the ass)
loc = './pr_lab1_2020-21_data/digits'
waves,ids,digits = data_parser(loc)
mfccs, deltas, ddeltas = extract_features(waves)

def find_indices(list,item):
    indices = []
    for idx, value in enumerate(list):
        if value == item:
            indices.append(idx)
    return indices

def find_mfccs(mfccs,ids,n1=1,n2=2):
    ## find indices of speakers
    n1_indices = find_indices(ids,n1)
    n2_indices = find_indices(ids,n2)
    
    n1_mffcs = []
    for i in n1_indices:
        n1_mffcs.append(mfccs[i][0])
        n1_mffcs.append(mfccs[i][1])
        
    n2_mffcs = []
    for i in n2_indices:
        n2_mffcs.append(mfccs[i][0])
        n2_mffcs.append(mfccs[i][1])
        
    plt.hist(np.array(n1_mffcs))
    plt.show()
    plt.hist(np.array(n2_mffcs))
    plt.show()
    
    return n1_mffcs, n2_mffcs
    
    
def extract_mfsc(waves,ids,n1,n2):
    n1_indices = find_indices(ids,n1)
    n2_indices = find_indices(ids,n2)
    

#find_mfccs(mfccs,ids,n1=1,n2=2)

Extracting MFCC features...:   0%|          | 0/133 [00:00<?, ?it/s]

Extracting MFCC features...: 100%|██████████| 133/133 [00:00<00:00, 141.00it/s]


Feature extraction completed: 13 MFCC per frame.


In [165]:
### STEP 5 ###
loc = './pr_lab1_2020-21_data/digits'
waves,ids,digits = data_parser(loc)
mfccs, deltas, ddeltas = extract_features(waves)

def concat_features(feat1,feat2,feat3):
    res = []
    for m,d,dd in zip(feat1,feat2,feat3):
        res.append(np.concatenate((m,d,dd),axis=1))
    return res

res = concat_features(mfccs,deltas,ddeltas)

Extracting MFCC features...:   0%|          | 0/133 [00:00<?, ?it/s]/home/dimitris/.local/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
Extracting MFCC features...: 100%|██████████| 133/133 [00:00<00:00, 212.56it/s]


Feature extraction completed: 13 MFCC per frame.


[array([[-7.9974536e+02,  5.2436073e+01,  4.4898477e+00, ...,
          4.1649427e-02,  4.1649427e-02,  4.1649427e-02],
        [-7.6516626e+02,  5.9436680e+01, -1.5254484e+01, ...,
         -3.0728260e-01, -3.0728260e-01, -3.0728260e-01],
        [-7.4390204e+02,  7.6625160e+01, -2.0886837e+01, ...,
         -7.1273953e-01, -7.1273953e-01, -7.1273953e-01],
        ...,
        [-7.7027325e+02,  5.2412197e+01, -1.4215784e+01, ...,
         -1.7892337e+00, -1.7892337e+00, -1.7892337e+00],
        [-7.4858636e+02,  7.2358536e+01, -1.8634851e+01, ...,
          5.7213098e-01,  5.7213098e-01,  5.7213098e-01],
        [-7.5479022e+02,  7.7437370e+01, -3.7469754e+00, ...,
         -4.8865414e-01, -4.8865414e-01, -4.8865414e-01]], dtype=float32),
 array([[-7.93298584e+02,  8.90612488e+01, -5.94499016e+00, ...,
         -1.57914627e+00, -1.57914627e+00, -1.57914627e+00],
        [-7.48459900e+02,  9.88377380e+01, -1.87612991e+01, ...,
         -2.04236555e+00, -2.04236555e+00, -2.04236555e+00]